In [1]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [3]:
from jetbot import Camera

#camera = Camera.instance(width=224, height=224, fps=10)
camera = Camera.instance(width=300, height=300)

In [4]:
detections = model(camera.value)

while str(detections) == '[[]]':
    print("no dectection")
    time.sleep(1)
    detections = model(camera.value)
    
print(detections)

[[{'label': 53, 'confidence': 0.849846601486206, 'bbox': [0.3198620080947876, 0.35256868600845337, 0.5846240520477295, 0.7367245554924011]}]]


In [5]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()
detections_widget.value = str(detections)
display(detections_widget)

Textarea(value="[[{'label': 53, 'confidence': 0.849846601486206, 'bbox': [0.3198620080947876, 0.35256868600845…

In [6]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

{'label': 53, 'confidence': 0.849846601486206, 'bbox': [0.3198620080947876, 0.35256868600845337, 0.5846240520477295, 0.7367245554924011]}


In [7]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    #Image zoomed to 224,224 versus 224,244 obstacle avoidance model
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [8]:
import PID

global follow_speed
follow_speed = Speed

global turn_gain
turn_gain = 1.5

global follow_speed_pid, follow_spped_pid_model
follow_speed_pid_model = 1

follow_speed_pid = PID.PositionalPID(1.5,0,0.05)

global turn_gain_pid
turn_gain_pid = PID.PositionalPID(0.15,0,0.05)

global object_yservo_pid
object_yersvo_pid = PID.PositionalPID(3,0.5,0)

NameError: name 'Speed' is not defined

In [9]:
from jetbot import Robot

robot = Robot()

In [10]:
from jetbot import bgr8_to_jpeg

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Calculate the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Calculate the length of a two-dimensional vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Find the detection closest to the center of the image"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    image = change['new']
    # Calculate all detected objects
    detections = model(image)
    
    # Draw all detected objects on the image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    
    # Select to match the detection of the selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # Let the detection be closest to the center of the field and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    # If don't detected object,jetbot car will keep advance
    if det is None:
        pass
        robot.forward(float(speed_widget.value))
        
    # If detected object,control Jetbot follow object
    else:
        # Move the robot forward and control the x distance between the proportional target and the center
        center = detection_center(det)
        robot.set_motors(
            float(speed_widget.value + turn_gain_widget.value * center[0]),
            float(speed_widget.value - turn_gain_widget.value * center[0])
        )
    
    # Update image display to widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

In [11]:
#camera.unobserve_all()
#camera.observe(execute, names='value')

import threading
import inspect
import ctypes
''' definition of the close thread function'''
def _async_raise(tid, exctype):
  """raises the exception, performs cleanup if needed"""
  tid = ctypes.c_long(tid)
  if not inspect.isclass(exctype):
    exctype = type(exctype)
  res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
  if res == 0:
    raise ValueError("invalid thread id")
  elif res != 1:
    # """if it returns a number greater than one, you're in trouble,
    # and you should call it again with exc=NULL to revert the effect"""
    ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
    raise SystemError("PyThreadState_SetAsyncExc failed")
def stop_thread(thread):
  _async_raise(thread.ident, SystemExit)
'''The function of thread 1, which continuously calls the detection function'''
def test():
    while True:
        execute({'new': camera.value})        

thread1 = threading.Thread(target=test)
thread1.setDaemon(False)
thread1.start()


In [12]:
import time
stop_thread(thread1)
camera.unobserve_all()
time.sleep(1.0)
robot.stop()